# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [2]:
import glob

import numpy as np
import scipy as sp
import scipy.io
from scipy import signal

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ground_truth = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1)
    print(np.shape(ground_truth))
    
    # Load data using LoadTroikaDataFile
    # ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    # Compute pulse rate estimates and estimation confidence.
    fs = 125
    window_length = 8*fs
    window_shift = 2*fs
    # Bandpass filter
    ppg  = bandpass(ppg, fs)
    accx = bandpass(accx, fs)
    accy = bandpass(accy, fs)
    accz = bandpass(accz, fs)

    # Magnitude of acceleration as 3D
    acc = np.sqrt(accx**2 + accy**2 + accz**2)

    bpm_predictions = []
    list_confidence = []

    for i in range(0, len(ppg)- window_length +1 , window_shift):
        ppg_current = ppg[i: i+ window_length]
        acc_current = acc[i: i+ window_length]

        # Apply Fourrier Transform
        ppg_freqs, ppg_ffts = fourrier_transformation(ppg_current, fs)
        acc_freqs, acc_ffts = fourrier_transformation(acc_current, fs)
        
        # Remove unwanted frequencies 
        ppg_ffts[ppg_freqs <= 40/60] = 0.0
        ppg_ffts[ppg_freqs >= 240/60] = 0.0
        acc_ffts[acc_freqs <= 40/60] = 0.0
        acc_ffts[acc_freqs >= 240/60] = 0.0


        # Get frequency that has the largerst FFT coefficient
        ppg_freq = ppg_freqs[np.argmax(ppg_ffts, axis=0)]  
        acc_freq = acc_freqs[np.argmax(acc_ffts, axis=0)]

        confidence = calculate_confidence(ppg_freqs, ppg_ffts, ppg_freq)
        
                
        k = 1
        while abs(ppg_freq - acc_freq) <= 0.01 and k <=3:                
            ppg_freq   = ppg_freqs[np.argsort(ppg_ffts, axis=0)[-k]]
            acc_freq = acc_freqs[np.argsort(acc_ffts, axis=0)[-k]]
            confidence = calculate_confidence(ppg_freqs, ppg_ffts, ppg_freq)
            
            k += 1
            

        bpm_predictions.append(ppg_freq * 60) # Convert to bpm from Hz
        list_confidence.append(confidence)

    errors = np.abs(np.subtract(bpm_predictions, ground_truth))



        # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
        #errors, confidence = np.ones(100), np.ones(100)  # Dummy placeholders. Remove
    return errors, list_confidence


def bandpass (signals, fs):
    """
    Filter pulse rate that will be restricted in bandpass 
    
    """
    # here we assumes pulse rate will be restricted between 40BPM (beats per minute) and 240BPM
    pass_band=(40/60.0, 240/60.0)
    b, a = sp.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signals)

def fourrier_transformation(signals, fs):   
    
    freqs = np.fft.rfftfreq(len(signals), 1/fs)
    fft = np.abs(np.fft.rfft(signals, len(signals)))
    return freqs, fft

def calculate_confidence(freq, fft, fulse_rate, windows = 1):
    
    """
    Inputs:
    freq: np.array Frequency bins
    fft: np.array  Magnitude of FFT 
    fulse_rate: the pulse rate
    windows: 
    Returns:
        confidence: float.
    """
    frequency_spectrum = (freq > fulse_rate - windows) & (freq <fulse_rate + windows)
    
    return np.sum(fft[frequency_spectrum])/np.sum(fft)
# Evaluate()